In [1]:
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
# import plotly.graph_objects as go
from dash import dcc

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import sqlalchemy
from os import environ

import numpy as np
import pandas as pd

In [3]:
engine = sqlalchemy.create_engine("mariadb+mariadbconnector://"+environ.get("USER")+\
                                  ":"+environ.get("PSWD")+"@127.0.0.1:3306/nba")

In [4]:
A_cols = ['A.Name', 'PTS',
 'FGM', 'FGA', 'PM3', 'PA3', 'AST','Game_day']

A_cols = ", ".join(A_cols)

In [5]:
fields = "* "

inner_join =  "Box_scores INNER JOIN Players on Box_scores.Player_ID = Players.ID) A "

inner_select = "(SELECT * FROM " + inner_join 

outer_join = "INNER JOIN Teams on Teams.ID=A.Team_ID "

outer_select = "SELECT " + A_cols +  ", Teams.Name as Team FROM " + inner_select  + outer_join 
outer_select += ' ORDER BY Game_day DESC'

In [6]:
league = pd.read_sql(outer_select,engine,parse_dates=['Game_day'])

In [7]:
def map_conference(df):
    west = ['NOP','UTA', 'MEM','PHX', 'POR', 'SAC', 'SAS', 'OKC','DAL', 'DEN', 'GSW', 'HOU','LAC', 'LAL','MIN']
    
    df['Conference'] = 'west'
    df['Conference'].where(df['Team'].isin(west),'east',inplace=True)
    return df

In [8]:
def map_division(df):
    divisions = {}
    # eastern conference divisions
    divisions['atlantic'] = ['BOS','BKN','NYK','PHI','TOR']
    divisions['central'] = ['CHI','CLE','DET','IND','MIL']
    divisions['southeast'] = ['ATL','CHA','MIA','ORL','WAS']
    
    # western conference divisions
    divisions['northwest'] = ['DEN', 'MIN','OKC','POR','UTA']
    divisions['pacific'] = ['GSW','LAC','LAL','PHX','SAC']
    divisions['southwest'] = ['DAL','HOU','MEM','NOP','SAS']
    
    df['Division'] = 'west'
    
    for i in divisions:
        df['Division'].where(~df['Team'].isin(divisions[i]),i,inplace=True)
    return df

In [9]:
league = map_division(league)
league = map_conference(league)

In [10]:
def calculate_features(df):
    df['eFG'] = 100*(df['FGM']+.5*df['PM3'])/df['FGA']
    df['PM2'] = df['FGM'] - df['PM3']

    return df

In [11]:
league = calculate_features(league)

In [12]:
league_avg = league.dropna().mean(numeric_only=True)

In [13]:
div_avg = league.groupby('Division').mean()

In [14]:
det = league.loc[league['Team']=='DET']

In [15]:
conf_avg = league.groupby('Conference').mean()

In [16]:
det_avg = det.mean(numeric_only=True)

In [17]:
team_avg = league.groupby("Team").mean()

In [18]:
def teams(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

In [19]:
def slider():
    slider = dcc.Slider(
        id = 'n_players',
        min = 1,
        max = 15,
        value = 5,
        marks = {year: str(year) for year in range(1,16)},
        step = None,
        tooltip={'always_visible':False}
    )
    return slider

In [20]:
def opposition_select(opps, i, t):
    opts = teams(opps)
    d = dcc.Dropdown(options=opts,value=t,id=i,multi=False)
    
    return d

In [21]:
import dash_daq as daq

In [22]:
def update_df(opp,n=5):
    opps = league.loc[league['Team']==opp].groupby('Name',as_index=False).mean().sort_values('PTS',ascending=False)
    opps['Team'] = opp
    
    d = det.groupby('Name',as_index=False).mean().sort_values('PTS',ascending=False)
    d['Team'] = 'DET'
    
    df = pd.concat((d.head(n),opps.head(n)))
    
    df= df.round(1)
    
    return df

In [23]:
def toggle_names(df,player_lab=True):
    
    if(player_lab):
        fig = px.scatter(y="PM3", x="PM2",color="eFG",size = "AST",data_frame = df,symbol='Team',
                         labels=dict(PM3='Average 3PM',PM2='Average 2PM',eFG='eFG%'),range_x=(-.2,df['PM2'].max()+2),
                         text="Name", hover_data = ['PM3','PM2','eFG','AST','Name'],
                         color_continuous_scale='RdBu_r',range_y=(-.2,df['PM3'].max()+1))
        fig.update_traces(textposition='top center')

    else:
        fig = px.scatter(y="PM3", x="PM2",color="eFG",size = "AST",data_frame = df,symbol='Team',
                         labels=dict(PM3='Average 3PM',PM2='Average 2PM',eFG='eFG%'),range_x=(-.2,df['PM2'].max()+2),
                         hover_data = ['PM3','PM2','eFG','AST','Name'],
                         color_continuous_scale='RdBu_r',range_y=(-.2,df['PM3'].max()+1))
        
    return fig

In [24]:
def add_lines(fig,grouping='League'):
    avgs = {'League':league_avg}
    
    fig.add_hline(avgs[grouping]['PM3'],annotation_text = grouping+' average',
                  annotation_position='right top',line_dash='dash')
    fig.add_vline(avgs[grouping]['PM2'],annotation_text= grouping+' average',
                  annotation_position='right top',line_dash='dash')
    
    return fig

In [25]:
def left(opp_teams):
    d = html.Div([html.Label("Opposition: ", style = {'textAlign': 'center'}),
                    opposition_select(opp_teams,'drpdwn','ATL')])#'textAlign': 'left',
    
    return d

In [26]:
def right(t):
    d = html.Div([html.Label("Number of players for each team: ", style = {'textAlign': 'center'}),t],)
    return d

In [29]:
for i in league['Name'].unique():  
    league.loc[league['Name']==i,'Team'] = league.loc[league['Name']==i,'Team'].unique()[0]

In [33]:
league.columns

Index(['Name', 'PTS', 'FGM', 'FGA', 'PM3', 'PA3', 'AST', 'Game_day', 'Team',
       'Division', 'Conference', 'eFG', 'PM2'],
      dtype='object')

In [35]:
league.to_csv('../../Pistons_dash.csv',index=False,columns=['Name', 'PTS', 'FGM', 'FGA', 'PM3', 'PA3', 'AST', 'Team'])

In [ ]:
def drop_players(league):

In [27]:
opps = league.loc[league['Team']=='ATL'].groupby('Name',as_index=False).mean().sort_values('PTS',ascending=False)
opps['Team'] = 'ATL'
d = det.groupby('Name',as_index=False).mean().sort_values('PTS',ascending=False)
d['Team'] = 'DET'
df = pd.concat((d.head(5),opps.head(5)))
df= df.round(1)

fig = toggle_names(df)
fig.update_layout(title ="Bubble size = Average AST",
                  title_x = 0.5,coloraxis_colorbar_x=1.15)



fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_traces(textposition='top center')

fig = add_lines(fig)

opp_teams = [i for i in league['Team'].unique() if i != "DET"]
opp_teams = sorted(opp_teams)

app = JupyterDash(__name__)

t = daq.ToggleSwitch(value=True, vertical=False,label='Toggle player labels',id='player_lab',size=32)

buttons_style = dict(width='45%',display='inline-block',height='5%')
app.layout = html.Div(className="p-heading",
                      children=[html.H1(children="Detroit Pistons field goal production"),
                                html.Div(children=[html.Div(children=[left(opp_teams)],
                                                    style=buttons_style),
                                                    html.Div(children=[right(slider())],
                                                     style=buttons_style)],
                                         style={'height':'24%'}),
                                html.Div([t, dcc.Graph(figure = fig, id = 'graph')])
])

@app.callback(
    Output('graph','figure'),
    Input('drpdwn','value'),
    Input('player_lab','value'),
    Input('n_players','value'))
def update_figure(opp,player_lab,n):
    
    df = update_df(opp,n)

    fig = toggle_names(df,player_lab)
    
    fig.update_layout(title ="Bubble size = Average AST",title_x = 0.5,
                      coloraxis_colorbar_x=1.15)

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig = add_lines(fig)

    
    return fig

In [28]:
app.run_server(mode = "external")

Dash app running on http://127.0.0.1:8050/


In [29]:
league.head()

,Name,PTS,FGM,FGA,PM3,PA3,AST,Game_day,Team,Division,Conference,eFG,PM2
0,Payton Pritchard,14,5,8,4,7,3,2022-04-03,BOS,atlantic,east,87.500000,1
1,Grant Williams,16,6,7,4,5,0,2022-04-03,BOS,atlantic,east,114.285714,2
2,Derrick White,17,6,7,3,3,3,2022-04-03,BOS,atlantic,east,107.142857,3
3,Jaylen Brown,32,12,17,4,5,5,2022-04-03,BOS,atlantic,east,82.352941,8
4,Al Horford,10,4,9,2,6,6,2022-04-03,BOS,atlantic,east,55.555556,2


In [31]:
league[league.columns[0:8]].to_csv('Pistons_dash.csv',index=False)

In [ ]:
# fig.add_scatter(y=other["PM3"], x=other["PM2"],color=other["eFG"],size = other["AST"])#,
#                 color_continuous_scale='Blues')

# for k in df['Team'].unique():
#     temp = df.loc[df['Team']==k].sort_values("PTS",ascending=False)
#     for i in range(10):
#         fig.add_annotation(text=temp.index[i],x=temp.loc[temp.index[i],'PM2'],
#                            y=temp.loc[temp.index[i],'PM3'])


# fig = go.Figure()

# # Team averages
# temp = det.groupby('Name').mean()
# opps = league.loc[league['Team']=='BKN'].groupby('Name').mean()

# # size limit definition
# upper_limit = max(temp['AST'].max(),opps['AST'].max())
# sizeref = 2*upper_limit/100
# sizemin = 10

# colorbar=dict(tickvals = list(range(0,100,10)),ticktext=list(range(0,100,10)))

# # plot DET stats
# marker = dict(size=temp['AST'],sizeref=sizeref,sizemin=sizemin,
#               color=temp['eFG'],colorscale='Blues',showscale=True,colorbar=colorbar)

# fig.add_trace(go.Scatter(y=temp["PM3"], x=temp["PM2"],mode='markers',
#                          name='DET',marker=marker))

# # plot opposition stats
# marker = dict(size=opps['AST'],sizeref=sizeref,sizemin=sizemin,colorbar=colorbar,
#               color=opps['eFG'],colorscale='Reds',showscale=True,colorbar_x=1.15)

# fig.add_trace(go.Scatter(y=opps["PM3"], x=opps["PM2"],mode='markers',name='POR',marker = marker))

# # plot league averages
# fig.add_hline(y=league_avg['PM3'])
# fig.add_vline(x=league_avg['PM2'])